**ShopAssist — Semantic Spotter**

A LangChain-based mini-project: content-based Product Recommendation + Conversational Agent + Data Analytics Agent + SQL Agent.


**Environment & Dependencies**


In [3]:
!pip -q install "langchain>=0.2.0"
!pip -q install "langchain-community>=0.2.0"
!pip -q install "langchain-openai>=0.1.0"
!pip -q install "langchain-classic>=1.0.0"
!pip -q install "langchain_experimental"
!pip -q install faiss-cpu
!pip -q install sqlalchemy
!pip -q install pandas
!pip -q install numpy
!pip -q install python-dotenv
!pip install -q langchain-openai
!pip install -q langchain-community

In [27]:
import os
import random
import numpy as np
import pandas as pd

from dotenv import load_dotenv
from sqlalchemy import create_engine

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent

from langchain_core.tools import Tool

Set your OpenAI API key. Replace 'YOUR_API_KEY_HERE' with your actual key.

In [32]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY_HERE" # IMPORTANT: Replace this with your actual OpenAI API key

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
embeddings = OpenAIEmbeddings()

**Load product and sales information from a CSV file.**

In [6]:
products_df = pd.read_csv("products.csv")
sales_df = pd.read_csv("sales.csv")

print("Products:", products_df.shape)
print("Sales:", sales_df.shape)



Products: (5000, 6)
Sales: (5000, 5)


**Create SQLite DB for SQL agent**

In [7]:
engine = create_engine("sqlite:///shop.db", echo=False)
products_df.to_sql("products", engine, if_exists="replace", index=False)
sales_df.to_sql("sales", engine, if_exists="replace", index=False)

db = SQLDatabase.from_uri("sqlite:///shop.db")

**Content-based product recommender (vector store) (cell)**

In [8]:
product_docs = []
for _, row in products_df.iterrows():
    text = f"{row['name']} | {row['category']} | {row['description']} | {row['tags']}"
    meta = {
        "product_id": int(row["id"]),
        "name": row["name"],
        "category": row["category"],
        "price": float(row["price"]),
    }
    product_docs.append(Document(page_content=text, metadata=meta))

vector_store = FAISS.from_documents(product_docs, embeddings)
retriever = vector_store.as_retriever(search_kwargs={"k": 3})


In [9]:
def recommend_products(preference: str) -> str:
    """
    Content-based recommendation using semantic similarity.
    """
    results = retriever.invoke(preference)
    lines = []
    for doc in results:
        m = doc.metadata
        lines.append(
            f"- {m['name']} (Category: {m['category']}, Price: ${m['price']:.2f})"
        )
    if not lines:
        return "No matching products found."
    return "Here are some recommended products:\n" + "\n".join(lines)


def get_product_details(product_name: str) -> str:
    """
    Lookup details of a product by exact name (case-insensitive).
    """
    matches = products_df[products_df["name"].str.lower() == product_name.lower()]
    if matches.empty:
        return f"No product found with name '{product_name}'."
    row = matches.iloc[0]
    return (
        f"Product: {row['name']}\n"
        f"Category: {row['category']}\n"
        f"Price: ${row['price']:.2f}\n"
        f"Description: {row['description']}\n"
        f"Tags: {row['tags']}"
    )

** Wrap as LangChain tools for use by an agent**

In [10]:
recommender_tool = Tool(
    name="product_recommender",
    func=recommend_products,
    description=(
        "Recommend products based on user preferences, "
        "such as 'wireless headphones for gaming under 200 dollars'."
    ),
)

product_info_tool = Tool(
    name="product_info_lookup",
    func=get_product_details,
    description="Get detailed information about a product given its exact name.",
)


**Conversational ShopAssist agent (cell)**

In [11]:
def shopassist_core(user_query: str) -> str:
    """
    Simple reasoning + tool routing without deprecated Agent APIs.
    """

    lower_q = user_query.lower()

    # 1. Use recommender if the user asks for suggestions
    if any(k in lower_q for k in ["recommend", "suggest", "looking for", "need", "find me"]):
        recs = recommend_products(user_query)
        prompt = (
            "You are ShopAssist, a helpful shopping assistant.\n"
            "User request:\n"
            f"{user_query}\n\n"
            "Tool output (product recommendations):\n"
            f"{recs}\n\n"
            "Convert this into a concise, friendly response for the user."
        )
        return llm.invoke(prompt).content

    # 2. If query mentions a known product name
    #   (limit to first N names to avoid scanning very large catalogs every time)
    for name in products_df["name"].head(500):
        if name.lower() in lower_q:
            details = get_product_details(name)
            prompt = (
                "You are ShopAssist, a helpful shopping assistant.\n"
                "User request:\n"
                f"{user_query}\n\n"
                "Tool output (product details):\n"
                f"{details}\n\n"
                "Explain this product clearly and concisely."
            )
            return llm.invoke(prompt).content

    # 3. Fallback: generic conversational answer
    generic_prompt = (
        "You are ShopAssist, an AI shopping assistant. "
        "Answer the user's question in a concise, helpful way.\n\n"
        f"User: {user_query}"
    )
    return llm.invoke(generic_prompt).content


def chat_with_shop_assist(user_query: str) -> str:
    return shopassist_core(user_query)


In [13]:
print(chat_with_shop_assist("I need comfortable running shoes for daily jogging."))
print()
print(chat_with_shop_assist("Suggest some wireless gaming headphones under 2000 INR."))


Sure! Here are some comfortable running shoes for your daily jogging:

1. **Sportswear Item 3645** - $25.02
2. **Sportswear Item 4624** - $270.88
3. **Sportswear Item 4005** - $166.13

Let me know if you need more information on any of these!

Sure! Here are some wireless gaming headphones you might like, all under 2000 INR:

1. **Electronics Item 2998** - $70.93
2. **Electronics Item 3812** - $49.89
3. **Electronics Item 3904** - $163.80

Let me know if you need more information on any of these!


**sales_df already loaded above from CSV**

In [14]:
pandas_agent = create_pandas_dataframe_agent(
    llm,
    sales_df,
    verbose=True,
    allow_dangerous_code=True
)

** Pandas Data Analytics agent (cell)**

In [15]:
def ask_analytics(question: str) -> str:
    """
    Natural language analytics over the sales DataFrame.
    """
    res = pandas_agent.invoke({"input": question})
    if isinstance(res, dict) and "output" in res:
        return res["output"]
    return str(res)


**SQL agent (cell)**

In [20]:
sql_agent = create_sql_agent(
    llm=llm,
    db=db,
    verbose=True,
    agent_type="zero-shot-react-description", # Use string literal instead of AgentType
)

In [21]:
def ask_sql(question: str) -> str:
    """
    Ask questions that require SQL reasoning over the products and sales tables.
    Example: 'List the top 2 most expensive products and their categories.'
    """
    return sql_agent.run(question)

In [22]:
print(ask_sql("What are the top 2 most expensive products and their categories?"))
print()
print(ask_sql("For each category, what is the average product price?"))


/tmp/ipython-input-2087860338.py:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  return sql_agent.run(question)




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ""  products, salesI need to check the schema of the "products" table to find out the relevant columns for product names, prices, and categories.  
Action: sql_db_schema  
Action Input: "products"  
CREATE TABLE products (
	id BIGINT, 
	name TEXT, 
	category TEXT, 
	description TEXT, 
	price FLOAT, 
	tags TEXT
)

/*
3 rows from products table:
id	name	category	description	price	tags
1	Home Item 1	Home	A high-quality home product, model 1, designed for everyday use.	234.45	kitchen,eco-friendly,decor
2	Sportswear Item 2	Sportswear	A high-quality sportswear product, model 2, designed for everyday use.	137.27	breathable,running,training
3	Sportswear Item 3	Sportswear	A high-quality sportswear product, model 3, designed for everyday use.	258.99	outdoor,lightweight,running
*/I have the necessary information about the "products" table. I will now construct a query to find the top 2 most expensive products 

**Simple menu / orchestrator (cell)**

In [23]:
def run_demo():
    print("=== ShopAssist Semantic Spotter Demo ===")
    print("1. Conversational Shopping Assistant")
    print("2. Data Analytics Agent (Pandas)")
    print("3. SQL Agent")
    choice = input("Choose mode (1/2/3): ").strip()

    if choice == "1":
        query = input("You: ")
        print("\nShopAssist:", chat_with_shop_assist(query))
    elif choice == "2":
        query = input("Analytics question: ")
        print("\nAnalytics Agent:", ask_analytics(query))
    elif choice == "3":
        query = input("SQL question: ")
        print("\nSQL Agent:", ask_sql(query))
    else:
        print("Invalid choice.")



In [25]:
def run_chatbot():
    """
    Simple console chatbot loop.
    Type 'exit', 'quit', or 'q' to stop.
    """
    print("=== ShopAssist Chatbot ===")
    print("Ask me anything about products, recommendations, or sales.")
    print("Type 'exit' to quit.\n")

    while True:
        user_input = input("You: ").strip()
        if user_input.lower() in {"exit", "quit", "q"}:
            print("ShopAssist: Goodbye!")
            break

        answer = chat_with_shop_assist(user_input)
        print("ShopAssist:", answer)
        print()


In [31]:
run_chatbot()


=== ShopAssist Chatbot ===
Ask me anything about products, recommendations, or sales.
Type 'exit' to quit.

You: Suggest some wireless headphone under 5000 INR
ShopAssist: Sure! Here are some great wireless headphones you can find under 5000 INR:

1. **Electronics Item 527** - ₹287.94
2. **Electronics Item 502** - ₹267.06
3. **Electronics Item 625** - ₹293.60

Let me know if you need more information on any of these!

You: exit
ShopAssist: Goodbye!
